In [2]:
!pip install pandas
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
df1 = pd.read_csv('house-prices-advanced-regression-techniques/train.csv', encoding='utf-8')
df2 = pd.read_csv('house-prices-advanced-regression-techniques/test.csv', encoding='utf-8')
df = pd.concat([df1, df2], axis=0, ignore_index=True)
print("dimension(row, col):", df.shape)

print("preview")
print(df.head())

print("data type and non-null value:")
print(df.info())

missing = df.isnull().sum().sort_values(ascending=False)
print("null value:")
print(missing[missing > 0])

print("stastical features:")
print(df.describe())

dimension(row, col): (2919, 81)
preview
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCond

In [ ]:
import numpy as np

# Step 2: Missing Value Handling

missing_ratio = df.isnull().mean().sort_values(ascending=False)

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

categorical_cols = df.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

# Drop columns with very high missing rate (>50%)
high_missing_cols = missing_ratio[missing_ratio > 0.5].index
df.drop(columns=high_missing_cols, inplace=True)

print("Missing values after imputation/dropping:")
print(df.isnull().sum()[df.isnull().sum() > 0])
print("dimension(row, col):", df.shape)



Missing values after imputation/dropping:
Series([], dtype: int64)
dimension(row, col): (1460, 76)


In [ ]:
ordinal_levels = {'Po', 'Fa', 'TA', 'Gd', 'Ex'}

#Scan all object-dtype columns and pick those whose unique values ⊆ ordinal_levels
ordinal_candidates = []
for col in df.select_dtypes(include=['object']).columns:
    vals = set(df[col].dropna().unique())
    if vals.issubset(ordinal_levels):
        ordinal_candidates.append(col)

print("Columns with values only in", ordinal_levels, ":\n", ordinal_candidates)



KeyError: 'BuyerID'

In [28]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder

# Step 4: Feature Encoding

ordinal_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
}

for col in ordinal_candidates:
    df[col] = df[col].map(ordinal_mapping)

remaining_cats = [
    c for c in df.select_dtypes(include=['object']).columns
    if c not in ordinal_candidates
]
df = pd.get_dummies(df, columns=remaining_cats, drop_first=True)

print("DataFrame shape after mapping:", df.shape)


#Simulate buyer preferences and encode
n_samples = df.shape[0]
n_buyers = 100  # arbitrary number of unique buyers
np.random.seed(42)
df['BuyerID'] = np.random.choice(np.arange(n_buyers), size=n_samples)
df['SizePreference'] = np.random.choice(['Small','Large'], size=n_samples)
df['NewOldPreference'] = np.random.choice(['Old','New'], size=n_samples)

# Encode BuyerID as integer labels (for random‐effects design matrix)
le = LabelEncoder()
df['BuyerID_enc'] = le.fit_transform(df['BuyerID'])

# Binary encode size and new/old preferences
df['SizePref_enc']    = df['SizePreference'].map({'Small':0, 'Large':1})
df['NewOldPref_enc']  = df['NewOldPreference'].map({'Old':0,   'New':1})

# (Optionally) drop the original categorical columns
df.drop(columns=['BuyerID','SizePreference','NewOldPreference'], inplace=True)

print("After encoding, dataframe shape:", df.shape)


Defaulting to user installation because normal site-packages is not writeable
DataFrame shape after mapping: (2919, 223)
After encoding, dataframe shape: (2919, 226)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:

from sklearn.preprocessing import StandardScaler

# Step 5: Feature Scaling

numeric_cols = df.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print(df[numeric_cols].head())



         Id  MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  \
0 -1.735986    0.150409    -0.126156 -0.189854     0.705529    -0.529518   
1 -1.731260    0.150409     0.038149  0.580944     0.705529    -0.529518   
2 -1.726533    0.150409     0.914439  1.409551     1.466540    -0.529518   
3 -1.721807   -0.840925     0.421526  0.259961     1.466540    -0.529518   
4 -1.712354   -0.840925     0.147685  0.567179    -0.816494    -0.529518   

   YearBuilt  YearRemodAdd  MasVnrArea  ExterQual  ...  3SsnPorch  \
0   0.960424      0.818268    0.875277        NaN  ...  -0.031404   
1   0.891548      0.770023    0.613739        NaN  ...  -0.031404   
2   0.857110      0.673532    2.059890        NaN  ...  -0.031404   
3   0.994862      0.914759    0.798354        NaN  ...  -0.031404   
4  -0.348212     -1.015052   -0.632412        NaN  ...  -0.031404   

   ScreenPorch  PoolArea   MiscVal    MoSold    YrSold  SalePrice  \
0    -0.207884       0.0 -0.145801 -1.586507  0.138684   0.

C:\Users\xxyyffyeah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\xxyyffyeah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\xxyyffyeah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [29]:
# Seperate train dataset and test dataset
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['BuyerID_enc'], random_state=42)
